企业名称划分，使测试集中出现的企业不在训练集中出现，用训练出来的模型预测新的实体（两个未知是否竞争关系的企业）之间的关系

In [2]:
#读取企业名称（及其简称）文件
import pandas as pd
file = open('../OriginalDataSet/CompanyName.txt')
CompanyName=pd.read_csv(file)
file.close()

In [3]:
CompanyName

,company
0,深宝 深圳市深宝实业股份有限公司 深深宝
1,深圳市康达尔(集团)股份有限公司 康达尔 康达 中科创业
2,深圳市农产品股份有限公司 农产品
3,广弘控股 广东广弘控股股份有限公司 美雅 粤美雅
4,川老窖 泸州老窖 老窖 泸州老窖股份有限公司
5,古井贡 古井贡酒 古井 安徽古井贡酒股份有限公司
6,金德发展 金德 庆云发展 株庆云 西王食品 株洲庆云 西王食品股份有限公司 西王
7,正虹饲料 湖南正虹科技发展股份有限公司 正虹 正虹科技
8,丰乐种业 丰乐 合肥丰乐种业股份有限公司
9,南方控股 南方黑芝麻集团股份有限公司 南控股 南方食品 广西斯壮 南控 黑芝麻


In [4]:
#企业名称总数
Num_Companys=len(CompanyName)
#Num_Companys=CompanyName.shape[0]
Num_Companys

176

In [6]:
#每一行表示同一个企业的名称，每行按空格进行切分,并按照名称长度从大到小进行排序，认为长度最大的名称为企业全称
def cut_names(name):
    start = 0
    i = 0
    names = []

    punt_list = ' '
    for word in name:
        if word in punt_list and token not in punt_list: #检查标点符号下一个字符是否还是标点
            names.append(name[start:i+1])
            start = i+1
            i += 1
        else:
            i += 1
            token = list(name[start:i+2]).pop() # 取下一个字符
    if start < len(name):
        names.append(name[start:])
    return names

Companys_Names=[]
for i in range(len(CompanyName)):
    names=cut_names(CompanyName.company[i])
    for j in range(len(names)):
        names[j]=names[j].replace(' ','')
    names.sort(key = lambda name:len(name),reverse=True)#按名称长度排序，倒叙，从大到小
    Companys_Names.append(names)

In [7]:
Companys_Names

[['深圳市深宝实业股份有限公司', '深深宝', '深宝'],
 ['深圳市康达尔(集团)股份有限公司', '中科创业', '康达尔', '康达'],
 ['深圳市农产品股份有限公司', '农产品'],
 ['广东广弘控股股份有限公司', '广弘控股', '粤美雅', '美雅'],
 ['泸州老窖股份有限公司', '泸州老窖', '川老窖', '老窖'],
 ['安徽古井贡酒股份有限公司', '古井贡酒', '古井贡', '古井'],
 ['西王食品股份有限公司', '金德发展', '庆云发展', '西王食品', '株洲庆云', '株庆云', '金德', '西王'],
 ['湖南正虹科技发展股份有限公司', '正虹饲料', '正虹科技', '正虹'],
 ['合肥丰乐种业股份有限公司', '丰乐种业', '丰乐'],
 ['南方黑芝麻集团股份有限公司', '南方控股', '南方食品', '广西斯壮', '南控股', '黑芝麻', '南控'],
 ['北京燕京啤酒股份有限公司', '燕京啤酒', '燕啤', '燕京'],
 ['罗牛山股份有限公司', '罗牛山', '罗牛'],
 ['西藏银河科技发展股份有限公司', '西藏发展', '拉萨啤酒', '藏发展'],
 ['中水集团远洋股份有限公司', '中水渔业', '中水'],
 ['酒鬼酒股份有限公司', '湘酒鬼', '酒鬼酒', '酒鬼'],
 ['广西贵糖(集团)股份有限公司', '贵糖股份', '贵糖'],
 ['河北承德露露股份有限公司', '承德露露', '露露'],
 ['宜宾五粮液股份有限公司', '五粮液'],
 ['北京顺鑫农业股份有限公司', '顺鑫农业', '顺鑫'],
 ['烟台张裕葡萄酿酒股份有限公司', '张裕'],
 ['新希望六和股份有限公司', '新希望'],
 ['河南双汇投资发展股份有限公司', '双汇发展', '双汇实业', '双汇'],
 ['南宁糖业股份有限公司', '南宁糖业', '南糖'],
 ['兰州黄河企业股份有限公司', '兰州黄河', '兰黄河'],
 ['中粮生物化学(安徽)股份有限公司', '中粮生化', '丰原生化', '丰原'],
 ['中基健康产业股份有限公司', '中基健康', '新中基'],
 ['甘肃皇台酒业股份有限公司', '皇台酒业',

In [8]:
file = open('CompanysNames20181026.txt', 'w')
for i in range(len(Companys_Names)):
    for j in range(len(Companys_Names[i])):
        file.write(Companys_Names[i][j]+' ')
    file.write('\n')
file.close()

In [12]:
CompanysNames=[]
file = open('CompanysNames20181026.txt')
for line in file.readlines():
    line=line.strip()
    names=line.split(' ')
    CompanysNames.append(names)

In [13]:
CompanysNames

[['深圳市深宝实业股份有限公司', '深深宝', '深宝'],
 ['深圳市康达尔(集团)股份有限公司', '中科创业', '康达尔', '康达'],
 ['深圳市农产品股份有限公司', '农产品'],
 ['广东广弘控股股份有限公司', '广弘控股', '粤美雅', '美雅'],
 ['泸州老窖股份有限公司', '泸州老窖', '川老窖', '老窖'],
 ['安徽古井贡酒股份有限公司', '古井贡酒', '古井贡', '古井'],
 ['西王食品股份有限公司', '金德发展', '庆云发展', '西王食品', '株洲庆云', '株庆云', '金德', '西王'],
 ['湖南正虹科技发展股份有限公司', '正虹饲料', '正虹科技', '正虹'],
 ['合肥丰乐种业股份有限公司', '丰乐种业', '丰乐'],
 ['南方黑芝麻集团股份有限公司', '南方控股', '南方食品', '广西斯壮', '南控股', '黑芝麻', '南控'],
 ['北京燕京啤酒股份有限公司', '燕京啤酒', '燕啤', '燕京'],
 ['罗牛山股份有限公司', '罗牛山', '罗牛'],
 ['西藏银河科技发展股份有限公司', '西藏发展', '拉萨啤酒', '藏发展'],
 ['中水集团远洋股份有限公司', '中水渔业', '中水'],
 ['酒鬼酒股份有限公司', '湘酒鬼', '酒鬼酒', '酒鬼'],
 ['广西贵糖(集团)股份有限公司', '贵糖股份', '贵糖'],
 ['河北承德露露股份有限公司', '承德露露', '露露'],
 ['宜宾五粮液股份有限公司', '五粮液'],
 ['北京顺鑫农业股份有限公司', '顺鑫农业', '顺鑫'],
 ['烟台张裕葡萄酿酒股份有限公司', '张裕'],
 ['新希望六和股份有限公司', '新希望'],
 ['河南双汇投资发展股份有限公司', '双汇发展', '双汇实业', '双汇'],
 ['南宁糖业股份有限公司', '南宁糖业', '南糖'],
 ['兰州黄河企业股份有限公司', '兰州黄河', '兰黄河'],
 ['中粮生物化学(安徽)股份有限公司', '中粮生化', '丰原生化', '丰原'],
 ['中基健康产业股份有限公司', '中基健康', '新中基'],
 ['甘肃皇台酒业股份有限公司', '皇台酒业',